# Градиентный бустинг

Градиентный спуск (Gradient Descent)

Это метод оптимизации: ты берёшь параметры одной модели (например, веса нейросети) и двигаешься по градиенту вниз, чтобы уменьшить ошибку.

Ассоциация: спуск с горы — каждый шаг ты делаешь туда, где “ниже” (меньше loss).

# Градиентный спуск 

Градиентный бустинг

Это модель/алгоритм ансамбля, который тоже использует идею градиента, но не по весам одной модели, а в “пространстве функций”: мы добавляем новые деревья как “маленькие поправки”, уменьшающие loss. Это классическое описание у Фридмана.

Ассоциация:
представь, что ты пишешь текст, а потом его по очереди правят 100 редакторов.
1-й редактор исправил очевидные ошибки, 2-й — то, что осталось, 3-й — ещё тоньше… В итоге получается сильный результат.

# Отличие бустинга от спуска

мы строим не одну “большую умную” модель, а ансамбль из многих простых моделей (обычно небольших деревьев).
Модели добавляются по очереди: каждая новая пытается исправить ошибки предыдущих.

XGBoost и CatBoost библиотеки которые осущ град бустинг на деревьях

больше инфы: 

**XGBoost**

Это библиотека, которая “делает градиентный бустинг на деревьях” очень эффективно (скорость/оптимизации/масштабирование). В их документации прямо сказано, что XGBoost — это “optimized distributed gradient boosting library”, и что это бустинг в рамках Gradient Boosting framework.

Ассоциация: “градиентный бустинг на стероидах” — тот же алгоритм по идее, но очень круто оптимизированный.

**CatBoost**

Это тоже градиентный бустинг на деревьях, но сделанный так, чтобы очень хорошо работать с категориальными признаками (типа “город”, “пол”, “профессия”) и уменьшать переобучение/утечки при их обработке. Это прямо заявлено в их статье и документации.

Ассоциация: XGBoost = “быстрый мощный бустинг”, CatBoost = “бустинг, который особенно удобен, когда много категорий”.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier


# 1) Загружаем Titanic (OpenML)
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

# y может быть строками ('0'/'1') — приведём к int 0/1
y = y.astype(int)

# 2) Разделим на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3) Препроцессинг: числовые и категориальные колонки
num_cols = X_train.select_dtypes(include=["number"]).columns
cat_cols = X_train.select_dtypes(exclude=["number"]).columns

numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop"
)

# 4) Baseline: Logistic Regression
baseline = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=2000))
])

# 5) Gradient Boosting (быстрый вариант в sklearn)
gb = Pipeline(steps=[
    ("prep", preprocess),
    ("model", HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_depth=3,
        max_iter=400,
        early_stopping=True,
        random_state=42
    ))
])

# 6) Обучаем
baseline.fit(X_train, y_train)
gb.fit(X_train, y_train)

# 7) Оцениваем
for name, model in [("Baseline(LogReg)", baseline), ("GradientBoosting(HistGB)", gb)]:
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)
    f1w = f1_score(y_test, pred, average="weighted")
    print("\n===", name, "===")
    print("Accuracy:", round(acc, 4))
    print("F1 weighted:", round(f1w, 4))
    print(classification_report(y_test, pred, digits=4))


/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:212: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11


=== Baseline(LogReg) ===
Accuracy: 0.9504
F1 weighted: 0.9502
              precision    recall  f1-score   support

           0     0.9515    0.9691    0.9602       162
           1     0.9485    0.9200    0.9340       100

    accuracy                         0.9504       262
   macro avg     0.9500    0.9446    0.9471       262
weighted avg     0.9503    0.9504    0.9502       262


=== GradientBoosting(HistGB) ===
Accuracy: 0.9466
F1 weighted: 0.9463
              precision    recall  f1-score   support

           0     0.9458    0.9691    0.9573       162
           1     0.9479    0.9100    0.9286       100

    accuracy                         0.9466       262
   macro avg     0.9468    0.9396    0.9429       262
weighted avg     0.9466    0.9466    0.9463       262



/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:227: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:227: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:227: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [7]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
"""
#загружаем датасет сердце с опенМЛ
X, y = fetch_openml("heart-disease", version=1, as_frame=True, return_X_y = True)

#типируем все значения метки как инт
y = y.astype(int)
"""
#загружем датасет
from sklearn.preprocessing import LabelEncoder

data = fetch_openml("heart-disease", version=1, as_frame=True)  # БЕЗ return_X_y
df = data.frame  # тут уже X + y в одной таблице

print("target_names:", data.target_names)
print("columns:", df.columns.tolist())

# выбираем колонку-таргет (часто она называется "class")
target_col = None

if data.target_names:
    for t in data.target_names:
        if t in df.columns:
            target_col = t
            break

if target_col is None:
    candidates = ["target", "num", "class", "label", "y", "diagnosis", "outcome"]
    col_map = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in col_map:
            target_col = col_map[c.lower()]
            break

if target_col is None and getattr(data, "target", None) is not None:
    y = pd.Series(data.target)
    X = df
else:
    y = df[target_col]
    X = df.drop(columns=[target_col])

# превращаем y в 0/1
le = LabelEncoder()
y = le.fit_transform(pd.Series(y).astype(str).str.strip())



#разделяем данные на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42, stratify=y
)

#разделяем колонки с числами и с признаками
num_cols = X_train.select_dtypes(include = ["number"]).columns
cat_cols = X_train.select_dtypes(exclude = ["number"]).columns

#если не находим нужное число заменяем на стреднее (крч фильтр грубо говоря)
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy = "median"))
])

#если не находим нужный признак то заменяем на самый частый прзнак 
categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

#теперь пропускаем все числа/признаки через фильтры для них
# готовим данные к модельки, фильтруем их/проверяем, (заполнение пропусков + one-hot)
preprocess = ColumnTransformer(
    transformers = [
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ]
)

# модель 1: лог регрессия, предсказание метрики
baseline = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=2000)) #классификатор
])

#модель 2: град бустинг
gb = Pipeline(steps=[
    ("prep", preprocess),
    ("model", HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_depth=3,
        max_iter=400,
        early_stopping=True,
        random_state=42
    ))
])

#обучение модели
baseline.fit(X_train, y_train)
gb.fit(X_train, y_train)

#сравниваем результаты двух моделек: гб и регрессия
for name, model in [("Baseline(LogReg)", baseline), ("GradientBoosting(HistGB)", gb)]:
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)
    f1w = f1_score(y_test, pred, average="weighted")
    print("\n===", name, "===")
    print("Accuracy:", round(acc, 4))
    print("F1 weighted:", round(f1w, 4))
    print(classification_report(y_test, pred, digits=4))


target_names: []
columns: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

=== Baseline(LogReg) ===
Accuracy: 0.8033
F1 weighted: 0.7997
              precision    recall  f1-score   support

           0     0.8636    0.6786    0.7600        28
           1     0.7692    0.9091    0.8333        33

    accuracy                         0.8033        61
   macro avg     0.8164    0.7938    0.7967        61
weighted avg     0.8126    0.8033    0.7997        61


=== GradientBoosting(HistGB) ===
Accuracy: 0.8197
F1 weighted: 0.8172
              precision    recall  f1-score   support

           0     0.8696    0.7143    0.7843        28
           1     0.7895    0.9091    0.8451        33

    accuracy                         0.8197        61
   macro avg     0.8295    0.8117    0.8147        61
weighted avg     0.8262    0.8197    0.8172        61



/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:333: RuntimeWarning: divide by zero encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-p

In [5]:
data = fetch_openml("heart-disease", version=1, as_frame=True)
df = data.frame

print("data.target_names =", data.target_names)
print("df columns =", df.columns.tolist())


data.target_names = []
df columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
